In [108]:
import pandas as pd
import sqlite3 as sql
import requests
import re
import json
from bs4 import BeautifulSoup as bs

In [109]:
r=requests.get("https://www.eventshigh.com/")
c=r.content
conn=sql.connect("Events.db")
cur=conn.cursor()

In [110]:
soup=bs(c,"html.parser")
cur.execute("CREATE TABLE IF NOT EXISTS int_url (id INTEGER PRIMARY KEY, Event URL TEXT, Name TEXT, Description TEXT, Start DateTime TEXT, End DateTime TEXT, Location TEXT)")
cur.execute("SELECT * FROM int_url")
cur.execute("CREATE UNIQUE INDEX int_url_Name on int_url ( Name )")
print(*cur.fetchall(), sep='\n')

In [111]:
int_url = []
non_int_url = []
for link in soup.find_all('a'):
    str1=str(link.get('href'))
    if str1.startswith('/detail/') and len(int_url)<10 and "https://www.eventshigh.com"+str1 not in int_url:
        int_url.append("https://www.eventshigh.com"+str1)
    elif len(int_url)==10:
        break
    else:
        non_int_url.append("https://www.eventshigh.com"+str1)

In [112]:
df=pd.DataFrame()
df["Non Interesting URLs"]=pd.Series(non_int_url)
df["Interesting URLs"]=pd.Series(int_url)
df

,Non Interesting URLs,Interesting URLs
0,https://www.eventshigh.com/bangalore/adventure...,https://www.eventshigh.com/detail/Delhi/41d8ae...
1,https://www.eventshigh.com/bangalore/attractio...,https://www.eventshigh.com/detail/Bangalore/06...
2,https://www.eventshigh.com/bangalore/classes+a...,https://www.eventshigh.com/detail/Bangalore/3c...
3,https://www.eventshigh.com/bangalore/concerts+...,https://www.eventshigh.com/detail/Bangalore/69...
4,https://www.eventshigh.com/bangalore/food?src=...,https://www.eventshigh.com/detail/Bangalore/99...
5,https://www.eventshigh.com/bangalore/fun+and+g...,https://www.eventshigh.com/detail/Bangalore/00...
6,https://www.eventshigh.com/bangalore/health+an...,https://www.eventshigh.com/detail/Bangalore/c6...
7,https://www.eventshigh.com/bangalore/nature+an...,https://www.eventshigh.com/detail/Bangalore/f6...
8,https://www.eventshigh.com/bangalore/parties+a...,https://www.eventshigh.com/detail/bangalore/f9...
9,https://www.eventshigh.com/bangalore/sightseei...,https://www.eventshigh.com/detail/bangalore/39...


In [115]:
""" Name, By, Description, Daybeg, Dayend, Location, Price """
l= []
for url in int_url:
    d= {}
    r1=requests.get(url)
    c1=r1.content
    page_soup = bs(c1,'html.parser')
    data = page_soup.select("[type='application/ld+json']")[2]
    oJson = json.loads(data.text)
    product=oJson[0]
    d["Event URL"]=url
    d["Name"]=bs(product['name']).text
    d["Description"]=product['description']
    d["Start Date"]=product['startDate']
    d["End Date"]=product['endDate']
    d["Location"]=bs(product['location']['name']+','+product['location']['address']).text
    try:
        cur.execute("INSERT INTO int_url VALUES ( NULL, ?, ?, ?, ?, ?, ?)",(d["Event URL"],d["Name"],d["Description"],d["Start Date"],d["End Date"],d["Location"]))
    except:
        pass
    l.append(d)

    df=pd.DataFrame(l)
df

,Event URL,Name,Description,Start Date,End Date,Location
0,https://www.eventshigh.com/detail/Delhi/41d8ae...,Chadar Trek - The frozen Zanskar River,CHADAR TREK \n Frozen River trek with the m...,2020-01-14T08:00:00+05:30,2020-01-20T08:00:00+05:30,"CHADAR TREK,Outside Delhi"
1,https://www.eventshigh.com/detail/Bangalore/06...,Forum Colorama 2019 - Forum Shantiniketan,"'A pen is mightier than the sword', they said!...",2019-11-24T12:00:00+05:30,2019-11-24T18:00:00+05:30,"The Forum Shantiniketan Mall,Whitefield Main R..."
2,https://www.eventshigh.com/detail/Bangalore/3c...,RCR and DR. A Live at XU,MTV #HipHopHustle fame rapper RCR performs 'LI...,2019-11-23T20:00:00+05:30,2019-11-24T01:00:00+05:30,"XU,23, Level C, The Leela Palace, Kodihalli, H..."
3,https://www.eventshigh.com/detail/Bangalore/69...,The Science Behind Data Visualization & Visua...,The Art of showing data in a visual context & ...,2019-11-17T17:00:00+05:30,2019-11-17T19:00:00+05:30,"Institute Of Product Leadership,No.1, KDP Buil..."
4,https://www.eventshigh.com/detail/Bangalore/99...,Understand any business in less than 15 minut...,"In today's business world, we all need basic f...",2019-11-23T16:30:00+05:30,2019-11-23T18:30:00+05:30,"CMRIT,132 AECS Layout,Kundalahalli ITPL Main R..."
5,https://www.eventshigh.com/detail/Bangalore/00...,Kanva Adventures & Night Camping,"Welcome to Weekend Adventure Camp, Presented b...",2019-11-11T17:00:00+05:30,2019-11-12T11:30:00+05:30,"Kanva Reservoir, SH 94, Ramanagara, Karnataka ..."
6,https://www.eventshigh.com/detail/Bangalore/c6...,Basics To Fondant (eggless),Skill level: Beginner \n Always intrigued by f...,2019-11-11T15:00:00+05:30,2019-12-12T18:00:00+05:30,"Flourish, School Of Baking Cake Decor,123-125..."
7,https://www.eventshigh.com/detail/Bangalore/f6...,Write Out Loud 190,Lahe Lahe brings to you an evening of soul on ...,2019-11-19T19:30:00+05:30,2019-11-19T21:30:00+05:30,"Lahe Lahe,Lahe Lahe, 2906 & 2907, 80 Feet Rd, ..."
8,https://www.eventshigh.com/detail/bangalore/f9...,Kodachadri Trek with Jeep Ride & Malpe Beach |...,Haelo Travelers! Let's hike up the amazing la...,2019-11-22T21:00:00+05:30,2019-11-25T05:00:00+05:30,"Kodachadri,Outside Bangalore"
9,https://www.eventshigh.com/detail/bangalore/39...,"Gokarna Chill at Private Beaches, Camping, Sta...",Upcoming Trips: \n \n \n \n Date \n Price ...,2019-11-15T21:00:00+05:30,2019-11-18T06:00:00+05:30,"Gokarna Private Beaches,Gokarna, Karnataka 581..."


In [116]:
cur.execute("SELECT * FROM int_url")
rows=cur.fetchall()
print(*rows, sep = "\n") 


(1, 'https://www.eventshigh.com/detail/Delhi/41d8ae86df50dd84f310241086f025b0-chadar-trek-the-frozen-zanskar?src=stream-home', 'Chadar Trek - The frozen Zanskar River', 'CHADAR TREK\xa0  \n  Frozen River trek with the most experienced team from Zanskar Village and Mumbai\'s trek heads "Mischief Treks"  \n \xa0 \n  Category  : Hiking \n \n  Terrain Grade:  Medium \n  Weather Conditions:  Challenging \n  Average Temperature:', '2020-01-14T08:00:00+05:30', '2020-01-20T08:00:00+05:30', 'CHADAR TREK,Outside Delhi')
(2, 'https://www.eventshigh.com/detail/Bangalore/0679be9ef2516cde919f9e579808c32a-forum-colorama-2019-forum-shantiniketan?src=stream-home', 'Forum Colorama 2019 - Forum Shantiniketan', "'A pen is mightier than the sword', they said! Well, it's time to inculcate a startling thought into the minds of these little geniuses with colorful crayons instead of pens! \n Sustainability is a term we hear quite often, but what does it mean?  Simply put, sustainability is", '2019-11-24T12:00:

In [117]:
conn.commit()
conn.close()

df.to_csv('eventshigh.csv')